In [ ]:
import pandas as pd
import numpy as np
import serial
import time
import math
import pandas
import tkinter
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import (FigureCanvasTkAgg)
from scipy.signal import savgol_filter, medfilt
from scipy.ndimage import gaussian_filter
from skimage.measure import ransac, LineModelND

In [ ]:
def connectionToRPLIDAR(port: str, rate: int):
    serial_port = port
    baud_rate = rate

    ser = serial.Serial(serial_port, baud_rate)
    
    if ser.isOpen(): print("Serial connection is open. RPLidar is connected.")
    else: print("Something is wrong with serial. Please check your connection.")

def startScan():
    if ser.isOpen():
        SCAN_DURATION_SECONDS = 30
        SECOND_SCAN_DELAY_SECONDS = 20
        SECOND_SCAN_DURATION_SECONDS = 30

        data1 = []
        data2 = []

        timeout_reached = False
        start_time = time.time()

        print("Performing the first scan...")
        while not timeout_reached:
            serial_line = ser.readline().decode().strip()
            if serial_line == 'First scan completed.':
                timeout_reached = True
            else:
                data1.append(serial_line)
    
            if time.time() - start_time >= SCAN_DURATION_SECONDS:
                timeout_reached = True

        print("First scan completed.")

        print(f"Waiting for {SECOND_SCAN_DELAY_SECONDS} seconds before the second scan...")
        time.sleep(SECOND_SCAN_DELAY_SECONDS)

        print("Performing the second scan...")
        timeout_reached = False
        start_time = time.time()

        while not timeout_reached:
            serial_line = ser.readline().decode().strip()
            if serial_line == 'Second scan completed.':
                timeout_reached = True
            else:
                data2.append(serial_line)

            if time.time() - start_time >= SECOND_SCAN_DURATION_SECONDS:
                timeout_reached = True

        print("Second scan completed.")

        ser.close()
        #----------------------------------------------------------------------------------
        print("First 10 rows of first scan:")
        data1[:10]

        print("First 10 rows of second scan:")
        data2[:10]
        
        return data1, data2
        
    else: 
        print("No serial connection. Please check your connection.")
        
        return [], []
    
def preprocessing(data: str):
    df = pandas.DataFrame(data, columns=['Angle', 'Distance', 'Quality'])
    df = df.astype({'Angle': float,'Distance': float, 'Quality': int })
    
    #converting angles from degrees into radians
    df.insert(1, "Angle(radius)", df['Angle'] * (math.pi / 180.0), True)
    
    #converting into cartesian coordinates
    df.insert(4, "X", df['Distance'] * df['Angle(radius)'].apply(lambda x: math.cos(x)), True)
    df.insert(5, "Y", df['Distance'] * df['Angle(radius)'].apply(lambda x: math.sin(x)), True)
    
    #displaying graph of scanning points
    root = tkinter.Tk()
    root.title('Point cloud from scanning')
    root.geometry("500x500") 
    
    figure = plt.Figure(figsize=(5, 5), dpi=100)
    ax = figure.add_subplot(111)
    ax.scatter(df['X'], df['Y'], s=10, c='blue')
    scatter = FigureCanvasTkAgg(figure, root)
    scatter.get_tk_widget().pack()
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    root.mainloop() 
    
    return df

def movingAverage(data: [], rolling_number: int):
    X_SMA = data['X'].rolling(rolling_number).mean()
    Y_SMA = data['Y'].rolling(rolling_number).mean()
    
    #deleting NaN values
    X_SMA = X_SMA[~np.isnan(X_SMA)]
    Y_SMA = Y_SMA[~np.isnan(Y_SMA)]
    
    #displaying graph
    root = tkinter.Tk()
    root.title('Moving Average algorithm on point cloud')
    root.geometry("500x500") 
    
    figure = plt.Figure(figsize=(5, 5), dpi=100)
    ax = figure.add_subplot(111)
    ax.scatter(data['X'], data['Y'], s=10, c='blue', alpha=0.3, label="Raw data")
    ax.scatter(X_SMA, Y_SMA, s=10, c='red', alpha=0.1, label="Moving Average filter")
    scatter = FigureCanvasTkAgg(figure, root)
    scatter.get_tk_widget().pack()
    ax.legend()
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    root.mainloop() 
    
    return X_SMA, Y_SMA

def SavitzkyGolayFilter(data: [], window_length: int):
    X_SGF = savgol_filter(data['X'], window_length=window_length, polyorder=2)
    Y_SGF = savgol_filter(data['Y'], window_length=window_length, polyorder=2)
    
    #displaying graph
    root = tkinter.Tk()
    root.title('Savitzky-Golay filter on point cloud')
    root.geometry("500x500") 
    
    figure = plt.Figure(figsize=(5, 5), dpi=100)
    ax = figure.add_subplot(111)
    ax.scatter(df['X'], df['Y'], s=10, c='blue', alpha=0.3, label="Raw data")
    ax.scatter(X_SGF, Y_SGF, c='r', s=10, alpha=0.1, label="Savitzky-Golay filter")
    scatter = FigureCanvasTkAgg(figure, root)
    scatter.get_tk_widget().pack()
    ax.legend()
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    root.mainloop() 
    
    return X_SGF, Y_SGF

def GaussianFilter(data: [], sigma: float):
    X_GF = gaussian_filter(data['X'], sigma=sigma)
    Y_GF = gaussian_filter(data['Y'], sigma=sigma)
    
    #displaying graph
    root = tkinter.Tk()
    root.title('Gaussian filter on point cloud')
    root.geometry("500x500") 
    
    figure = plt.Figure(figsize=(5, 5), dpi=100)
    ax = figure.add_subplot(111)
    ax.scatter(df['X'], df['Y'], s=10, c='blue', alpha=0.3, label="Raw data")
    ax.scatter(X_GF, Y_GF, c='r', s=10, alpha=0.1, label="Gaussian filter")
    scatter = FigureCanvasTkAgg(figure, root)
    scatter.get_tk_widget().pack()
    ax.legend()
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    root.mainloop() 
    
    return X_GF, Y_GF

def medianFilter(data: [], kernel_size: int):
    X_MF = medfilt(data['X'], kernel_size=kernel_size)
    Y_MF = medfilt(data['Y'], kernel_size=kernel_size)
    
    #displaying graph
    root = tkinter.Tk()
    root.title('Median filter on point cloud')
    root.geometry("500x500") 
    
    figure = plt.Figure(figsize=(5, 5), dpi=100)
    ax = figure.add_subplot(111)
    ax.scatter(df['X'], df['Y'], s=10, c='blue', alpha=0.3, label="Raw data")
    ax.scatter(X_MF, Y_MF, c='r', s=10, alpha=0.1, label="Median filter")
    scatter = FigureCanvasTkAgg(figure, root)
    scatter.get_tk_widget().pack()
    ax.legend()
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    root.mainloop() 
    
    return X_MF, Y_MF

def RANSAC_for_lines(min_pocet_vzoriek: int, max_vzdialenost: int, min_prilahlych_hodnot: int, hodnoty: []):
    min_pocet_vzoriek = min_pocet_vzoriek
    max_vzdialenost = max_vzdialenost
    min_prilahlych_hodnot = min_prilahlych_hodnot
    vysledne_hodnoty_X = []
    vysledne_hodnoty_Y = []
    zacinajuce_hodnoty = np.column_stack([hodnoty['X'], hodnoty['Y']])

    def ransac_algoritmus(vstupne_hodnoty: [], max_vzdialenost: int):
        model_robust, prilahle_hodnoty = ransac(vstupne_hodnoty, LineModelND, min_samples = min_pocet_vzoriek, 
                                   residual_threshold = max_vzdialenost, max_trials = 1000)
    
        prilahle_X = []
        prilahle_Y = []
        odlahle_X = []
        odlahle_Y = []
    
        for i in range(0,len(vstupne_hodnoty)):
            if (prilahle_hodnoty[i] == False):
                odlahle_X.append(vstupne_hodnoty[i][0])
                odlahle_Y.append(vstupne_hodnoty[i][1])
                continue

            prilahle_X.append(vstupne_hodnoty[i][0])
            prilahle_Y.append(vstupne_hodnoty[i][1])

        return prilahle_X, prilahle_Y, odlahle_X, odlahle_Y, model_robust

    for index in range(0,20):
        if (len(zacinajuce_hodnoty) <= min_pocet_vzoriek):
            print("Nedostatočný počet bodov.")
            break
            
        prilahle_hodnoty_X,prilahle_hodnoty_Y,odlahle_hodnoty_X,odlahle_hodnoty_Y,model = ransac_algoritmus(zacinajuce_hodnoty, max_vzdialenost = max_vzdialenost)
        
        if (len(prilahle_hodnoty_X) < min_prilahlych_hodnot):
            print("Nájdený dostatočný počet priľahlých hodnôt.")
            break
        
        zacinajuce_hodnoty = np.column_stack([odlahle_hodnoty_X, odlahle_hodnoty_Y])
        vysledne_hodnoty_X.append(prilahle_hodnoty_X)
        vysledne_hodnoty_Y.append(prilahle_hodnoty_Y)

    print("Našlo sa %d RANSAC čiar" % (len(vysledne_hodnoty_X)))

    root = tkinter.Tk()
    root.title('RANSAC algorithm for line fitting')
    root.geometry("500x500") 
    figure = plt.Figure(figsize=(5, 5), dpi=100)
    ax = figure.add_subplot(111)

    for i in range(0, len(vysledne_hodnoty_X)):
        ax.plot(vysledne_hodnoty_X[i], vysledne_hodnoty_Y[i])

    ax.scatter(hodnoty['X'], hodnoty['Y'], alpha=0.1)
    graph = FigureCanvasTkAgg(figure, root)
    graph.get_tk_widget().pack()
    ax.set_xlabel("X")
    ax.set_ylabel("Y")

    root.mainloop()
    
def RANSAC_for_planes(x: [], y: [], z: []):
    naj_plocha = None
    naj_prilahle_hodnoty = []
    naj_chyba = np.inf
    
    for _ in range(100):
        # Random nájsť 3 body
        body = np.random.choice(len(x), size=3, replace=False)
        p1 = np.array([x[body[0]], y[body[0]], z[body[0]]])
        p2 = np.array([x[body[1]], y[body[1]], z[body[1]]])
        p3 = np.array([x[body[2]], y[body[2]], z[body[2]]])

        # napasovanie plochy pre tieto 3 body pomocou metódy najmenších štvorcov
        v1 = p2 - p1
        v2 = p3 - p1
        normal = np.cross(v1, v2)
        normal /= np.linalg.norm(normal)
        d = -np.dot(normal, p1)
        
        # vzdialenosti od všetkých bodov ku ploche
        vzdialenosti = np.abs(np.dot(np.array([x, y, z]).T, normal) + d)
        
        # chyba pomocou súčtu štvorcov vzdialeností
        chyba = np.sum(vzdialenosti ** 2)
        
        # update modelu pokiaľ je chyba menšia ako predchádzajúca
        if chyba < naj_chyba:
            naj_plocha = normal, d
            naj_prilahle_hodnoty = np.where(vzdialenosti < 0.1)[0]
            naj_chyba = chyba
    #--------------------------------------------------------------------------------------
    root = tkinter.Tk()
    root.title('RANSAC algorithm for plane fitting')
    root.geometry("500x500") 
    figure = plt.Figure(figsize=(5, 5), dpi=100)
    ax = figure.add_subplot(111, projection='3d')
    ax.scatter(x, y, z, c='b', label='Zosnímané body')
    
    if naj_plocha is not None:
        if z[0] == 0:
            xx, yy = np.meshgrid(np.linspace(min(x), max(x), 10), np.linspace(min(y), max(y), 10))
            zz = (-normal[0] * xx - normal[1] * yy - d) / normal[2]
        else:
            xx, zz = np.meshgrid(np.linspace(min(x), max(x), 10), np.linspace(min(z), max(z), 10))
            yy = (-normal[0] * xx - normal[2] * zz - d) / normal[1]
        ax.plot_surface(xx, yy, zz, alpha=0.5, color='r', label='Nájdená plocha')

    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_zlabel('Z')
    ax.legend()
    graph = FigureCanvasTkAgg(figure, root)
    graph.get_tk_widget().pack()
    root.mainloop()
    #--------------------------------------------------------------------------------------
    if naj_plocha is not None and len(naj_prilahle_hodnoty) > 0:
        normal, d = naj_plocha
    
        x_plocha = x[naj_prilahle_hodnoty]
        if z[0] == 0:
            y_plocha = y[naj_prilahle_hodnoty]
        else: 
            y_plocha = z[naj_prilahle_hodnoty]
    
        min_x = np.min(x_plocha)
        max_x = np.max(x_plocha)
        min_y = np.min(y_plocha)
        max_y = np.max(y_plocha)
    
        velkost_x = max_x - min_x
        velkost_y = max_y - min_y
    
        print("Rozmery nájdenej plochy:")
        if z[0] == 0:
            print("Dĺžka [X]:", velkost_x)
            print("Šírka [Y]:", velkost_y)
        else:
            print("Šírka [X]:", velkost_x)
            print("Výška [Z]:", velkost_y)

        return velkost_x, velkost_y 
    
def generateDimensionsForUnity(save_path: str, width: float, height: float, length: float):
    output_data = pd.DataFrame({'X':[0], 'Y':[0], 'Z':[0], 'Width':[width/1000], 'Height':[length/1000], 'Rotation_X':[0], 'Rotation_Z':[0]}, columns=['X', 'Y', 'Z', 'Width', 'Height', 'Rotation_X', 'Rotation_Z'])
    output_data = pd.concat([output_data, pd.DataFrame({'X':[0], 'Y':[(height*10/2)/1000], 'Z':[(length*10/2)/1000], 'Width':[width/1000], 'Height':[height/1000], 'Rotation_X':[-90], 'Rotation_Z':[0]})], ignore_index=True)
    output_data = pd.concat([output_data, pd.DataFrame({'X':[0], 'Y':[(height*10/2)/1000], 'Z':[(-(length*10/2))/1000], 'Width':[width/1000], 'Height':[height/1000], 'Rotation_X':[90], 'Rotation_Z':[0]})], ignore_index=True)
    output_data = pd.concat([output_data, pd.DataFrame({'X':[(width*10/2)/1000], 'Y':[(height*10/2)/1000], 'Z':[0], 'Width':[height/1000], 'Height':[length/1000], 'Rotation_X':[0], 'Rotation_Z':[90]})], ignore_index=True)
    output_data = pd.concat([output_data, pd.DataFrame({'X':[(-(width*10/2))/1000], 'Y':[(height*10/2)/1000], 'Z':[0], 'Width':[height/1000], 'Height':[length/1000], 'Rotation_X':[0], 'Rotation_Z':[-90]})], ignore_index=True)
    print(output_data)
    np.savetxt(r''+save_path, output_data.values, fmt='%f', delimiter=';')